In [4]:
import requests
from  bs4 import BeautifulSoup
import pandas as pd
import datetime
import time
import progressbar


title = []
date = []
link = []
content = []


mydatetime = datetime.date(2016,1,1)
url_date = []
for k in range(0,366):
    date2016 = mydatetime+ datetime.timedelta(k)
    date2016 = str(date2016)
    date2016 = date2016.replace("-", "")
    url_date.append(date2016)

def scraping(turl, title, date, link, content):
    source_code = requests.get(turl)
    plain_text = source_code.content
    soup = BeautifulSoup(plain_text, 'lxml', from_encoding = 'utf-8')
    body = soup.find('div', class_ = 'list_body newsflash_body')
    yonhap = body.find('ul', class_ = 'type02')
    
    page_length = len(soup.find('div',class_='paging paging_ytn').find_all('a'))+2
    
    for x in range(1, page_length):
        source_code = requests.get(turl+"&page="+str(x))
        plain_text = source_code.content
        soup = BeautifulSoup(plain_text, 'lxml', from_encoding = 'utf-8')
        body = soup.find('div', class_ = 'list_body newsflash_body')
        yonhap = body.find('ul', class_ = 'type02')    
        
        for t in yonhap.find_all('strong'):
            if (t.get_text() != '동영상기사'):
                title.append(t.get_text())    
        
        for i in yonhap.find_all('span', class_ = 'date'):
            date.append(i.get_text())
        
        for j in yonhap.find_all('a', class_ = 'nclicks(fls.list)', href=True):
            link.append(j['href'])
    
with progressbar.ProgressBar(max_value = len(url_date)) as bar:
    for x in url_date:
        turl = "http://news.naver.com/main/list.nhn?sid2=140&sid1=001&oid=001&isYeonhapFlash=Y&mid=sec&mode=LPOD&date="+str(x)
        scraping(turl, title, date, link, content)
        #print(url_date.index(x) , "/" ,len(url_date) )
        bar.update(url_date.index(x))
    



100% (366 of 366) |###############################################################| Elapsed Time: 0:12:57 Time: 0:12:57


In [96]:
import random

data = pd.DataFrame({'title':title, 'date':date, 'link':link})
def scrap_contents(data, start, finish):
    with progressbar.ProgressBar(max_value=finish-start) as bar:
        for i, x in enumerate(data.link[start:finish]):
            url = x
            source_code = requests.get(url)
            plain_text = source_code.content
            soup = BeautifulSoup(plain_text, 'lxml', from_encoding='utf-8')

            article_content = soup.find('div', id="articleBodyContents")
            if (article_content != None):
                try:
                    for span in article_content.find_all('span'):
                        article_content.span.extract()
                except: pass
                try:
                    for script in article_content.find_all('script'):
                        article_content.script.extract()
                except: pass

                for a in article_content.find_all('a'):
                    article_content.a.extract()

                try:
                    article_text = article_content.get_text().split('=')[2]
                    data.content[i+start] = article_text
                except:
                    data.content[i+start] = "속보"
                    pass

            article_content2 = soup.find('div', id="articeBody")
            if (article_content2 != None):
                try:
                    for span in article_content2.find_all('span'):
                        article_content2.span.extract()
                except: pass
                try:
                    for script in article_content2.find_all('script'):
                        article_content2.script.extract()
                except: pass

                for a in article_content2.find_all('a'):
                    article_content2.a.extract()

                try:
                    article_text = article_content2.get_text().split('=')[2]
                    data.content[i+start] = article_text
                except:
                    data.content[i+start] = "속보"
                    pass
            article_content3 = soup.find('div', id="newsEndContents")
            if (article_content3 != None):
                try:
                    for span in article_content3.find_all('span'):
                        article_content3.span.extract()
                except:
                    pass
                try:
                    for script in article_content3.find_all('script'):
                        article_content3.script.extract()
                except:
                    pass

                for a in article_content3.find_all('a'):
                    article_content3.a.extract()

                try:
                    article_text = article_content3.get_text().split('=')[2]
                    data.content[i+start] = article_text
                except:
                    data.content[i+start] = "속보"
                    pass

            if (article_content == None and article_content2 == None and article_content3 == None):
                print(x, "예외 발생")
            time.sleep(1)
            bar.update(i)

In [153]:
#range 지정
a = range(29,32)
b = [j * 1000 for j in a]
#b[0] = 26890
b

[29000, 30000, 31000]

In [150]:
for i in b:
    scrap_contents(data, i, i+1000)
    time.sleep(120)
    print(i,"/31000 완료")



100% (1000 of 1000) |#############################################################| Elapsed Time: 0:14:31 Time: 0:14:31
N/A% (0 of 1000) |                                                               | Elapsed Time: 0:00:00 ETA:  --:--:--

29000 /31000 완료


100% (1000 of 1000) |#############################################################| Elapsed Time: 0:04:08 Time: 0:04:08
100% (1000 of 1000) |#############################################################| Elapsed Time: 0:00:00 Time: 0:00:00


30000 /31000 완료
31000 /31000 완료


In [149]:
#어디까지 했나 확인

import numpy as np

np.where(data.content)

(array([    0,     1,     2, ..., 29001, 29002, 29003], dtype=int64),)

In [151]:
data.to_csv("data_complete.csv")